# ![University Logo](https://redintercol.net/images/Logo_EAFIT.svg.png)
## **Trabajo 2**
### Mariel Viviana Sánchez
### Laura Riveros
### Maria del Rosario Castro Mantilla

#**Paso 3**

Para realizar la detección de tópicos en un conjunto de datos utilizando un enfoque basado en el aprendizaje no supervisado, como el modelo de Latent Dirichlet Allocation (LDA)

In [1]:
import gensim
from gensim.utils import simple_preprocess
from gensim import corpora
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
from gensim.models.ldamodel import LdaModel
from gensim.models import CoherenceModel

import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


#**Carga y limpieza de data**

In [2]:
!pip install pandas nltk gensim pyLDAvis

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 13.5 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import pandas as pd

In [7]:
# Load the CSV file
data = pd.read_csv('/content/drive/MyDrive/EAFIT/ALMACENAMIENTO Y RECUPERACIÓN INFO/TRABAJOS/TRABAJO 2/FINAL/climateTwitterData.csv')

data.head()

<ipython-input-7-82b08c36a835>:2: DtypeWarning: Columns (16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('/content/drive/MyDrive/EAFIT/ALMACENAMIENTO Y RECUPERACIÓN INFO/TRABAJOS/TRABAJO 2/FINAL/climateTwitterData.csv')


,Unnamed: 0.1,Unnamed: 0,id,author_id,text,retweets,permalink,date,formatted_date,favorites,mentions,hashtags,geo,urls,search_hashtags,location,sentiment1,sentiment2
0,0,0,1.211810e+18,7.590000e+17,"2020 is the year we #votethemout, the year we ...",15,https://twitter.com/Sphiamia/status/1211807074...,2019-12-31 00:31:35+00:00,Tue Dec 31 00:31:35 +0000 2019,46,NaN,#votethemout #climatestrike #rebelforlife,NaN,NaN,#climatestrike,"California, USA",negative,negative
1,1,1,1.210670e+18,2.219547e+07,Winter has not stopped this group of dedicated...,9,https://twitter.com/StephDujarric/status/12106...,2019-12-27 20:56:21+00:00,Fri Dec 27 20:56:21 +0000 2019,35,NaN,#climatefriday #climatestrike #ClimateAction,NaN,NaN,#climatestrike,"California, USA",positive,positive
2,2,2,1.210590e+18,1.070000e+18,WEEK 55 of #ClimateStrike at the @UN. Next wee...,545,https://twitter.com/AlexandriaV2005/status/121...,2019-12-27 15:50:22+00:00,Fri Dec 27 15:50:22 +0000 2019,3283,@UN @Fridays4future,#ClimateStrike,NaN,NaN,#climatestrike,"California, USA",positive,positive
3,3,3,1.210260e+18,1.339821e+09,"A year of resistance, as youth protests shape...",1,https://twitter.com/EnergyHouseVA/status/12102...,2019-12-26 17:53:26+00:00,Thu Dec 26 17:53:26 +0000 2019,2,NaN,#greta #gretathunberg #climatechange #fridaysf...,NaN,https://www.channelnewsasia.com/news/commentar...,#climatestrike,"California, USA",positive,positive
4,4,4,1.209640e+18,1.339821e+09,HAPPY HOLIDAYS #greta #gretathunberg #climate...,1,https://twitter.com/EnergyHouseVA/status/12096...,2019-12-25 00:56:37+00:00,Wed Dec 25 00:56:37 +0000 2019,4,NaN,#greta #gretathunberg #climatechange #fridaysf...,NaN,"http://www.energyhouse.us,http://www.pacenowfo...",#climatestrike,"California, USA",positive,positive


In [8]:
# Función de limpieza de texto
def limpiar_texto(tuit):
    tuit = re.sub(r'http\S+', '', tuit)  # Eliminar URLs
    tuit = re.sub(r'@\w+', '', tuit)     # Eliminar menciones
    tuit = re.sub(r'#\w+', '', tuit)     # Eliminar hashtags
    tuit = re.sub(r'\d+', '', tuit)      # Eliminar números
    tuit = re.sub(r'[^\w\s]', '', tuit)  # Eliminar puntuaciones
    tuit = tuit.lower()                  # Convertir a minúsculas
    tuit = ' '.join([word for word in tuit.split() if word not in stop_words])
    return tuit

# Aplicar la limpieza a la columna de tuits
data['tuit_limpio'] = data['text'].apply(limpiar_texto)

In [12]:
# Preprocesamiento
stop_words = stopwords.words('english')
lemmatizer = WordNetLemmatizer()

def preprocess(text):
    return [lemmatizer.lemmatize(token) for token in simple_preprocess(text) if token not in stop_words]

data['tuit_limpio'] = data['tuit_limpio'].apply(preprocess)

In [13]:
# Función para extraer hashtags
def extraer_hashtags(tuit):
    return re.findall(r'#(\w+)', tuit)

# Crear una nueva columna 'hashtags' con los hashtags extraídos
data['hashtags'] = data['text'].apply(extraer_hashtags)

# Asignar 'no_hashtag' a los tuits que no tienen hashtags
data['hashtags'] = data['hashtags'].apply(lambda x: x if len(x) > 0 else ['no_hashtag'])

#**K-Optimo con Gensim**

In [16]:
# Diccionario de todas las palabras
dictionary = corpora.Dictionary(data['tuit_limpio'])

#"bag-of-words"
corpus = [dictionary.doc2bow(text) for text in data['tuit_limpio']]
print(corpus[:5])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 4)], [(11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1)], [(3, 1), (10, 1), (12, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 2), (36, 1), (37, 1), (38, 1), (39, 1), (40, 2)], [(10, 1), (12, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 2)], [(49, 1), (50, 1), (51, 1), (52, 1), (53, 1)]]


In [17]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=1):
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = LdaModel(corpus=corpus, num_topics=num_topics, id2word=dictionary, random_state=42)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())
    return model_list, coherence_values

In [18]:
# Revisar el número de etiquetas únicas
unique_hashtags = data['search_hashtags'].nunique()
print(f"Número de hashtags únicos: {unique_hashtags}")

Número de hashtags únicos: 12


In [20]:
# K optimo
limit = 12  # Según el numero de etiquetas
model_list, coherence_values = compute_coherence_values(dictionary, corpus, data['tuit_limpio'], limit)


In [25]:
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " valor de Coherence", round(cv, 4))

Num Topics = 2  valor de Coherence 0.397
Num Topics = 4  valor de Coherence 0.3921
Num Topics = 6  valor de Coherence 0.3999
Num Topics = 8  valor de Coherence 0.3792
Num Topics = 10  valor de Coherence 0.4522


In [26]:
# Mejor modelo
optimal_model = model_list[coherence_values.index(max(coherence_values))]

# Mostrar los tópicos
topics = optimal_model.print_topics(num_words=10)
for topic in topics:
    print(topic)


(0, '0.032*"de" + 0.026*"le" + 0.020*"la" + 0.012*"en" + 0.007*"night" + 0.006*"un" + 0.006*"waste" + 0.005*"pa" + 0.005*"que" + 0.005*"el"')
(1, '0.015*"climate" + 0.013*"change" + 0.012*"need" + 0.009*"planet" + 0.008*"dont" + 0.008*"world" + 0.007*"people" + 0.007*"make" + 0.007*"like" + 0.007*"amp"')
(2, '0.020*"amp" + 0.019*"today" + 0.019*"climate" + 0.012*"people" + 0.010*"change" + 0.009*"great" + 0.009*"student" + 0.009*"young" + 0.008*"see" + 0.008*"thank"')
(3, '0.022*"climate" + 0.018*"u" + 0.016*"today" + 0.015*"capitol" + 0.013*"join" + 0.012*"school" + 0.012*"strike" + 0.011*"dc" + 0.009*"state" + 0.009*"action"')
(4, '0.017*"amp" + 0.008*"sydney" + 0.008*"house" + 0.008*"environment" + 0.008*"go" + 0.007*"australia" + 0.006*"u" + 0.006*"health" + 0.006*"business" + 0.005*"great"')
(5, '0.017*"new" + 0.014*"green" + 0.011*"talk" + 0.010*"time" + 0.008*"year" + 0.008*"amp" + 0.008*"like" + 0.007*"good" + 0.007*"deal" + 0.007*"still"')


#**K-Optimo con Nltk**

In [ ]:
# Convertir el texto en una matriz de cuentas (CountVectorizer)
count_vectorizer = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')
X = count_vectorizer.fit_transform(data['tuit_limpio'])

# Aplicar LDA para identificar los tópicos
lda = LatentDirichletAllocation(n_components=5, random_state=42)
lda.fit(X)

# Mostrar los términos principales de cada tópico
def mostrar_topicos(modelo, count_vectorizer, n_palabras):
    palabras = count_vectorizer.get_feature_names_out()
    for idx, topico in enumerate(modelo.components_):
        print(f"Tópico {idx}:")
        print([palabras[i] for i in topico.argsort()[-n_palabras:]])

mostrar_topicos(lda, count_vectorizer, 10)

Tópico 0:
['strike', 'great', 'need', 'united', 'energy', 'capitol', 'action', 'today', 'amp', 'climate']
Tópico 1:
['world', 'york', 'green', 'years', 'hall', 'dont', 'amp', 'earth', 'city', 'new']
Tópico 2:
['des', 'que', 'el', 'les', 'et', 'pour', 'le', 'en', 'amp', 'la']
Tópico 3:
['global', 'amp', 'future', 'action', 'young', 'world', 'people', 'today', 'change', 'climate']
Tópico 4:
['thank', 'global', 'save', 'dont', 'time', 'need', 'like', 'amp', 'planet', 'people']
